In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. 챗봇 말투로 파인튜닝

In [ ]:
# 파인튜닝 하려면
!pip uninstall konlpy
!pip uninstall pororo

In [ ]:
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 826.4/826.4 KB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 KB 56.9 MB/s eta 0:00:00


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import torch
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
# from pytorch_lightning.core.lightning import LightningModule
from pytorch_lightning import LightningModule
from torch.utils.data import DataLoader, Dataset
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import re

In [ ]:
from torch.utils.data import DataLoader, Dataset
from transformers import PreTrainedTokenizerFast

In [ ]:
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'

PAD = '<pad>'

In [ ]:
koGPT3_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/ko-gpt-trinity-1.2B-v0.5", bos_token=BOS, eos_token=EOS, unk_token="<unk>", pad_token=PAD, mask_token=MASK,)
model = GPT2LMHeadModel.from_pretrained('skt/ko-gpt-trinity-1.2B-v0.5')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
koGPT222_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token=BOS, eos_token=EOS, unk_token="<unk>", pad_token=PAD, mask_token=MASK,)
model222 = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
import urllib.request

urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv",
    filename="ChatBotData.csv",
)
Chatbot_Data = pd.read_csv("ChatBotData.csv")
# Test 용으로 300개 데이터만 처리한다.
Chatbot_Data = Chatbot_Data[:2000]
Chatbot_Data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [ ]:
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=50):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.q_token = Q_TKN
        self.a_token = A_TKN
        self.sent_token = SENT
        self.eos = EOS
        self.mask = MASK
        self.tokenizer = koGPT3_TOKENIZER

    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        turn = self._data.iloc[idx]
        q = turn["Q"]  # 질문을 가져온다.
        q = re.sub(r"([?.!,])", r" ", q)  # 구둣점들을 제거한다.

        a = turn["A"]  # 답변을 가져온다.
        a = re.sub(r"([?.!,])", r" ", a)  # 구둣점들을 제거한다.

        q_toked = self.tokenizer.tokenize(self.q_token + q + self.sent_token)
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)

        #질문의 길이가 최대길이보다 크면
        if q_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        #질문의 길이 + 답변의 길이가 최대길이보다 크면
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [self.mask,] * q_len + a_toked[1:]

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
        # 답변 labels을 index 로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변을 index 로 만든다.    
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        #질문+답변, 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)

In [ ]:
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=40):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.q_token = Q_TKN
        self.a_token = A_TKN
        self.sent_token = SENT
        self.eos = EOS
        self.mask = MASK
        self.tokenizer = koGPT222_TOKENIZER

    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        turn = self._data.iloc[idx]
        q = turn["Q"]  # 질문을 가져온다.
        q = re.sub(r"([?.!,])", r" ", q)  # 구둣점들을 제거한다.

        a = turn["A"]  # 답변을 가져온다.
        a = re.sub(r"([?.!,])", r" ", a)  # 구둣점들을 제거한다.

        q_toked = self.tokenizer.tokenize(self.q_token + q + self.sent_token)
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)

        #질문의 길이가 최대길이보다 크면
        if q_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        #질문의 길이 + 답변의 길이가 최대길이보다 크면
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [self.mask,] * q_len + a_toked[1:]

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
        # 답변 labels을 index 로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변을 index 로 만든다.    
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        #질문+답변, 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)

In [ ]:
def collate_batch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.cuda.LongTensor(data), torch.cuda.LongTensor(mask), torch.cuda.LongTensor(label)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_set = ChatbotDataset(Chatbot_Data, max_len=40)
#윈도우 환경에서 num_workers 는 무조건 0으로 지정, 리눅스에서는 2
train_dataloader = DataLoader(train_set, batch_size=32, num_workers=0, shuffle=True, collate_fn=collate_batch,)

In [ ]:
model.to(device)
model.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 1920)
    (wpe): Embedding(1024, 1920)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1920,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1920,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1920,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout)

In [ ]:
model222.to(device)
model222.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [ ]:
learning_rate = 3e-5
criterion = torch.nn.CrossEntropyLoss(reduction="none")
optimizer = torch.optim.Adam(model222.parameters(), lr=learning_rate)

epoch = 10
Sneg = -1e18

In [ ]:
# import gc

# gc.collect()

# torch.cuda.empty_cache()


In [ ]:
print ("start")
for epoch in range(epoch):
    for batch_idx, samples in enumerate(train_dataloader):
        optimizer.zero_grad()
        token_ids, mask, label = samples
        
        out = model222(token_ids)
        out = out.logits      #Returns a new tensor with the logit of the elements of input
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
        loss = criterion(mask_out.transpose(2, 1), label)
        # 평균 loss 만들기 avg_loss[0] / avg_loss[1] <- loss 정규화
        avg_loss = loss.sum() / mask.sum()
        avg_loss.backward()
        # 학습 끝
        optimizer.step()
print ("end")

start


<ipython-input-12-cecccbdfe2f9>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  return torch.cuda.LongTensor(data), torch.cuda.LongTensor(mask), torch.cuda.LongTensor(label)


end


In [ ]:
def bot( sent='0'):
  sent = '0'
  with torch.no_grad():
      while 1:
          word = input().strip()
          if word == "quit":
              break
          a = ""
          while 1:
              input_ids = torch.cuda.LongTensor(koGPT222_TOKENIZER.encode(Q_TKN + word + SENT + sent + A_TKN + a)).unsqueeze(dim=0)
              pred = model222(input_ids)
              pred = pred.logits
              gen = koGPT222_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().cpu().numpy().tolist())[-1]
              if gen == EOS:
                  break
              a += gen.replace("▁", " ")
          print("{}".format(a.strip()))
          break

In [ ]:
# torch.cuda.empty_cache()

In [ ]:
bot()

네! 덕분에 하루 잘 보내겠어요
잘가요


# 2. 유사도 기반 추천

In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 43.5 MB/s eta 0:00:00


In [ ]:
!pip install pororo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pororo-0.4.2-py3-none-any.whl (256 kB)
  Using cached sentence_transformers-2.2.2-py3-none-any.whl
  Using cached wget-3.2.zip (10 kB)
  Preparing metadata (setup.py) ... done
  Using cached torch-1.6.0-cp38-cp38-manylinux1_x86_64.whl (748.8 MB)
  Using cached torchvision-0.7.0-cp38-cp38-manylinux1_x86_64.whl (5.9 MB)
  Using cached word2word-1.0.0-py3-none-any.whl (31 kB)
  Using cached g2p_en-2.1.0-py3-none-any.whl (3.1 MB)
  Using cached Whoosh-2.7.4-py2.py3-none-any.whl (468 kB)
  Using cached marisa_trie-0.7.8-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
  Using cached kss-4.5.1.tar.gz (77 kB)
  Preparing metadata (setup.py) ... done
  Using cached fairseq-0.12.2-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.0 MB)
  Using cached hydra_core-1.0.7-py3-none-any.whl (123 kB)
  Using cached sacrebleu-2.3.1-py3-none-any.whl (118 kB)
  Using ca

In [ ]:
from pororo import Pororo
from sentence_transformers import util
import torch
import numpy as np
import urllib.request

In [ ]:
from tqdm import tqdm
tqdm.pandas()

In [ ]:
#크롤링한 데이터 업로드
import pandas as pd
df_res = pd.read_csv('/content/drive/MyDrive/NLP 스터디/final_resturaunt.csv')
df_date = pd.read_excel('/content/drive/MyDrive/NLP 스터디/데이트코스.xlsx')
df_drk = pd.read_excel('/content/drive/MyDrive/NLP 스터디/최종.xlsx')

In [ ]:
df_res.dropna(subset=['naver_blog_review_txt'], inplace=True)
df_date.dropna(subset=['특징'], inplace=True)
df_drk.dropna(subset=['특징'], inplace=True)

In [ ]:
# review에 대한 임베딩 열 추가
sTe = Pororo(task="sentence_embedding", lang="ko")
df_res['EmbVector'] = df_res['naver_blog_review_txt'].progress_map(lambda x : sTe(x))
df_res.head()

sTe = Pororo(task='sentence_embedding', lang='ko')
df_date['EmbVector'] = df_date['특징'].progress_map(lambda x : sTe(x))

sTe = Pororo(task='sentence_embedding', lang='ko')
df_drk['EmbVector'] = df_drk['특징'].progress_map(lambda x : sTe(x))




100%|██████████| 26/26 [00:00<00:00, 68.76it/s]


In [ ]:
# 특징을 통해 추천(특징과 리뷰의 코사인 유사도를 통해 추천)
def res(df, characteristic):
    emb = torch.tensor(df['EmbVector'].tolist())
    characteristic = sTe(characteristic)
    # 질문을 Tensor로 바꿉니다.
    characteristic = torch.tensor(characteristic)
    # 코사인 유사도 
    cos_sim = util.pytorch_cos_sim(characteristic, emb)
    #유사도가 가장 비슷한 질문 인덱스를 구합니다.
    best_sim_idx = int(np.argmax(cos_sim))
    # 질문의 유사도와 가장 비슷한  답변 제공
    name = df['name'][best_sim_idx]
    review = df['naver_blog_review_txt'][best_sim_idx]
    menu = df['cate_mix'][best_sim_idx]
    return name, menu, review

def date(df, characteristic):
    emb = torch.tensor(df['EmbVector'].tolist())
    characteristic = sTe(characteristic)
    # 질문을 Tensor로 바꿉니다.
    characteristic = torch.tensor(characteristic)
    # 코사인 유사도 
    cos_sim = util.pytorch_cos_sim(characteristic, emb)

    #유사도가 가장 비슷한 질문 인덱스를 구합니다.
    best_sim_idx = int(np.argmax(cos_sim))

    # 질문의 유사도와 가장 비슷한  답변 제공
    name = df['이름'][best_sim_idx]
    review = df['특징'][best_sim_idx]

    return name, review

def drk(df, characteristic):
    emb = torch.tensor(df['EmbVector'].tolist())
    characteristic = sTe(characteristic)
    # 질문을 Tensor로 바꿉니다.
    characteristic = torch.tensor(characteristic)
    # 코사인 유사도 
    cos_sim = util.pytorch_cos_sim(characteristic, emb)

    #유사도가 가장 비슷한 질문 인덱스를 구합니다.
    best_sim_idx = int(np.argmax(cos_sim))

    # 질문의 유사도와 가장 비슷한  답변 제공
    name = df['이름'][best_sim_idx]
    review = df['특징'][best_sim_idx]

    return name, review


# 3. 전체 합치기

In [ ]:
def find_location(input):
  if '안암' in input:
    return '안암'
  elif '혜화' in input:
    return '혜화'
  elif '성신' in input:
    return '성신'
  else:
    return None

print(find_location('안암동이 좋아요'))

안암


In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import re

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# prompt = '어떤 음식을 드시고 싶으세요?'
def kogpt2(prompt):
  with torch.no_grad():
    tokens = koGPT3_TOKENIZER.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
    gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=100)
    generated = koGPT3_TOKENIZER.batch_decode(gen_tokens)[0]
  return generated  

# kogpt2(prompt)

In [ ]:
loc = '안암'
print('어떤 식당에 가고싶으신가요?')
x = '맛있는 치킨을 파는 식당에 가서 치맥하고 싶어요'
data_res = df_res[df_res['dong'] == (loc+'동')]
data_res.reset_index(inplace=True)
name, menu, review = res(data_res, x)
review = review[:40]
prompt = '<'+menu+'>를 팔고 <'+review+'>한 리뷰를 받은<'+name+'>이라는 식당을 추천드립니다. 여기를 추천하는 이유는'
kogpt2(prompt)

어떤 식당에 가고싶으신가요?


'<닭 오리요리 후라이드 양념치킨 치킨 전문점>를 팔고 <항상 치킨은 이곳에서 먹어요ᄒᄒ 사장님 항상 친절하신 모습으로 반겨주시고>한 리뷰를 받은<치킨플러스>이라는 식당을 추천드립니다. 여기를 추천하는 이유는 이 곳이 강추하는 곳입니다. 치킨의 맛과 더불어 저렴하고 싱싱한 치킨을 맛볼 수 있기 때문입니다. 물론 가격대는 다른 곳보다 조금 높은 편이지만, 그만큼 맛이 좋으니까요. <닭 오리요리 후'

In [ ]:
review

'항상 치킨은 이곳에서 먹어요ㅎㅎ 사장님 항상 친절하신 모습으로 반겨주시고'

In [ ]:
#수정필요

def chat():
  print('안암 / 성신 / 혜화 중 어디에 가고 싶으신가요?')
  x = input()
  loc = find_location(x)
  while (loc == None):
    print('다시 선택해주세요')
    x = input()
    loc = find_location(x)
  print("'"+loc+"'"+'을 선택하였습니다.')

  print('드시고 싶은 메뉴, 방문하고자 하는 식당의 특징을 말씀해주세요.')
  x = input()
  data_res = df_res[df_res['dong'] == (loc+'동')]
  data_res.reset_index(inplace=True)
  name, menu, review = res(data_res, x)
  review = review[:40]
  prompt = '<'+menu+'>를 팔고 <'+review+'> 의 리뷰를 받은 <'+name+'> 이라는 식당을 추천드립니다. 여기를 추천하는 이유는'
  print(kogpt2(prompt))

  print('\n어떤 데이트를 하고 싶으신가요?')
  x = input()
  data_date = df_date[df_date['구역'] == loc]
  data_date.reset_index(inplace=True)
  name, review = date(data_date, x)
  # print(name, review, '한 데이트장소 추천드립니다.')
  prompt = '<'+review+'>의 특징이 있는 <'+name+'>이라는 코스를 추천드립니다. 여기를 추천하는 이유는'
  print(kogpt2(prompt))

  print('\n오늘 방문하고자 하는 술집의 원하는 분위기, 가격대 등이 있으면 말해주세요.')
  x = input()
  data_drk = df_drk[df_drk['구역'] == loc]
  data_drk.reset_index(inplace=True)
  name, review = drk(data_drk, x)
  prompt = '<'+review+'>의 특징이 있는 <'+name+'>이라는 술집을 추천드립니다. 여기를 추천하는 이유는'
  print(kogpt2(prompt))

  print('\n마음에 드셨나요?')

  return(bot())
  # return None

In [ ]:
chat()

안암 / 성신 / 혜화 중 선택
안암에서 놀게여
'안암'을 선택하였습니다.
어떤 식당에 가고싶으신가요?
샌드위치나 샐러드류
<제과제빵떡케익 제과점 제과점업>를 팔고 <샌드위치 5천원으로 최고의 갓성비템이라고 생각한다..아삭거리는 양상추가 > 의 리뷰를 받은 <디어브레드> 이라는 식당을 추천드립니다. 여기를 추천하는 이유는 가게가 굉장히 깔끔합니다. 특히 빵이 신선합니다. 가게에서 나오는 빵이 진열되어 있는 것이 보입니다. 제가 추천하는 메뉴는 빵 3종과 롤 종류 1종입니다. 가게에서 바로 구입이 가능

어떤 데이트장소에 가고싶으신가요?
디저트 맛있고 대화하기 좋은 카페요
<카페, 스콘, 디저트, 커피, 달콤한 간식, 책읽기, 대화>의 특징이 있는 <더보일>이라는 코스를 추천드립니다. 여기를 추천하는 이유는, 제가 가장 만족하는 코스가 스콘, 스콘, 디저트, 커피, 달콤한 간식, 책읽기, 대화, 그리고 대화로 이어지는 코스가 아닐까 합니다. <카페, 스콘, 디저트, 커피, 달콤한 간식, 책읽기, 대화>의 특징이 있는 <더보

오늘 방문하고자 하는 술집의 원하는 분위기, 가격대 등이 있으면 말해주세요.
단둘이 가볍게 한잔하기 좋은 곳이요
<식사류. 가성비. 조용함. 데이트.>의 특징이 있는 <56st>이라는 술집을 추천드립니다. 여기를 추천하는 이유는 정말 고급스러운 인테리어와 분위기 때문이 아니라 이 식당이 있는 곳이 바로 <카페>라는 점입니다. 보통 사람들이 <커피숍>이라고 알고 있는 그곳이 바로 이곳이랍니다. 하지만 이곳은 <카페>가 아닌 <식당>입니다. 이곳의 장점은 메뉴가 딱 3가지만 있다는 것입니다. 그 중

마음에 드셨나요?
넵ㅎㅎ 내일도 부탁해요
바쁠때 힘이 되어 주세요
